# The process of ETL begins..

# 1. EXTRACT phase 📤

Importing all the necessary libraries..

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime, timedelta

Importing and Extracting all the data chunks in various file formats which includes csv, xlsx, json and db respectively..

In [2]:
chunk1 = pd.read_csv("chunk_1.csv")
chunk2 = pd.read_excel("chunk_2.xlsx")
chunk3 = pd.read_json("chunk_3.json", orient="records", lines=True)
conn = sqlite3.connect("chunk_4.db")
chunk4 = pd.read_sql("SELECT * FROM invoices", conn)
conn.close()

Concatenating all the data chunks to make a single dataframe to work upon..

In [3]:
combined_df = pd.concat([chunk1, chunk2, chunk3, chunk4], ignore_index=True)
print("Combined DataFrame shape:", combined_df.shape)

Combined DataFrame shape: (310000, 23)


A glimpse of how the dummy dataset looks like..

In [4]:
combined_df.sample(5)

,invoice_id,invoice_date,product_id,product_name,product_category,price,quantity,payment_method,customer_id,customer_name,...,city,country,inventory_id,stock_available,restock_date,supplier_id,supplier_name,supplier_email,supplier_phone,payment_id
309582,efd6b598-e030-4826-9b07-063307d2c855,2025-05-22,e99209e7-9023-4a80-9077-15cdf66f4981,Develop Toys,Electronics,524.58,47.0,Debit Card,8db7cc7b-68bc-4afc-8e7e-32e437ec1622,Jordan Hurley,...,Michaelport,Nicaragua,d9e24e78-8a1e-49ca-b762-176b79b9a861,92.0,None,82764226-3914-4b4d-890a-90bd0e649427,Chapman-Sullivan,jason96@nguyen-rodriguez.com,4238197955334,659915741214
146732,NaN,2024-04-28 00:00:00,608c07e5-9400-4e49-963a-b48eec423ded,Great Groceries,Toys,2067.36,138.0,Bank Transfer,a7978ef9-40e1-46f8-8e12-116b3dba8e32,William Aguilar,...,Longton,Afghanistan,120c7ce5-410c-42f7-bfb3-d39da2a477f7,NaN,2025-07-08 00:00:00,7c5ea63c-6b49-4be7-aeba-2a394bb62817,Jones-Edwards,eric55@stafford.com,5448888828772.0,NaN
132038,50173a92-0bc3-4a35-96fb-70f2f7e11a36,2024-06-02 00:00:00,d43f6f83-a7ed-40b8-ae54-1271084942ea,Couple Electronics,Toys,5893.18,123.0,Bank Transfer,16a2898c-d541-48b3-9797-d63c164febff,Lisa Taylor,...,Angelaborough,Zambia,e1c12051-0ca5-4d94-96db-624b92aee656,46.0,2025-05-30 00:00:00,57f59ed2-1722-4454-9ae7-9dcba5066c2c,"Miller, Cobb and Porter",lopeztracy@gonzalez-williams.info,3618665141736.0,NaN
244178,2ffa55ef-e800-4d63-a0bb-5fa3f53337e3,2024-04-22,b4a7dafb-3e72-465e-b732-7dccb6161b89,Seven Clothing,Electronics,2470.17,2.0,Credit Card,487ac7e5-70cc-4da7-87c8-79e83abcffc9,Jessica Anderson,...,Sandraland,Bosnia and Herzegovina,489b63a4-3274-414a-84e4-c91ae0f53694,43.0,2025-07-19,22fe32b3-caca-4ed7-b111-1adb8fe266a9,"Mason, Hall and Charles",gmccullough@mcbride-white.net,9097624877286,350915792649
44856,bc4bd6c5-80a3-4060-b2a3-31722437cde7,2024-09-27,9364f0a2-c03d-4970-9ab4-1bdfa86d0518,Others Groceries,Clothing,1437.39,NaN,Cash,5f370f03-4d2b-4ad2-b349-e9d116bde5b1,Rebecca Little MD,...,Richardshire,Iraq,5c9a8149-75ea-494d-896d-86a72495edd3,35.0,2025-06-14,ff8a5092-249f-4f40-a06a-cbb539ce7217,Sawyer-Collier,epark@tucker-lee.com,9669843023394.0,NaN


# 2. TRANSFORM Phase🔄

Checking the duplicated rows and removing them as they won't be of any use in this case scenario..

In [5]:
combined_df.duplicated(keep=False).sum()

5164

😲 `AN ANOMALY`..
We have inserted 10K rows as duplicates while generating data. But, we are getting only 5164 rows as duplicates.WHY?

Key Reasons:

1. `Original Duplicates Exist`: Our source DataFrame may already contained some duplicate rows before operation and duplicated() counts all duplicates in the final dataset, not just your inserted ones.

2. `Overlap in Sampling`: When we sampled 10,000 rows with df.sample(n=10000), some of those rows might have already been duplicates of each other in the original data.

3. `Insertion Logic`: We inserted duplicates at random positions, but some original rows might have been identical to the duplicates inserted, causing them to merge in the count.

In [6]:
combined_df.drop_duplicates(keep='first', inplace=True)


What columns do we have?

In [7]:
combined_df.columns

Index(['invoice_id', 'invoice_date', 'product_id', 'product_name',
       'product_category', 'price', 'quantity', 'payment_method',
       'customer_id', 'customer_name', 'customer_age', 'customer_gender',
       'customer_email', 'city', 'country', 'inventory_id', 'stock_available',
       'restock_date', 'supplier_id', 'supplier_name', 'supplier_email',
       'supplier_phone', 'payment_id'],
      dtype='object')

Checking if there is any NULL values in all the columns of our dataset so that we can perform preprocessing to remove them.. 

In [8]:
combined_df.isnull().sum()

invoice_id           9211
invoice_date         9273
product_id           9203
product_name         9455
product_category     9317
price                9215
quantity             9188
payment_method       9337
customer_id          9331
customer_name        9183
customer_age         9321
customer_gender         0
customer_email          0
city                 9264
country              9126
inventory_id            0
stock_available      9294
restock_date         9229
supplier_id             0
supplier_name        9166
supplier_email          0
supplier_phone       9216
payment_id          63937
dtype: int64

Indeed!! there are a lot of NULL values in many of the columns. So let's preprocess them..

Critical identifiers like `invoice_id`, `customer_id`, `invoice_date` and `product_id`are usually essential. If these are missing, the rows might be invalid because you can't track transactions or customers. So, the first step would be to check if these columns are truly critical. If they are, dropping rows with nulls in these columns might be necessary resulting in nearly 5.62% loss from the dataset which is acceptable..

In [9]:
combined_df = combined_df.dropna(subset=['customer_id', 'invoice_id','invoice_date','product_id'])

Next, columns like `product_name`, `product_category`, `price`, and `quantity` and `customer_age` are related to the product details. Missing product names or categories could make analysis hard. So, These fields are critical for analysis and hence, we can perform necessary imputations Imputations to preserve data integrity.

In [10]:
combined_df['product_name'].fillna(combined_df['product_name'].mode()[0], inplace=True)
combined_df['product_category'].fillna(combined_df['product_category'].mode()[0], inplace=True)
combined_df['customer_age'].fillna(combined_df['customer_age'].mode()[0], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_6704\2511224472.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['product_name'].fillna(combined_df['product_name'].mode()[0], inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_6704\2511224472.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

In [11]:
combined_df['price'].fillna(combined_df['price'].median(), inplace=True)
combined_df['quantity'].fillna(combined_df['quantity'].median(), inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_6704\1894114866.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['price'].fillna(combined_df['price'].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_6704\1894114866.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

Customer information like `customer_name`, `email` is missing, but other details exist, so we might consider them as customers who might not have an email or not prefer to tell their name. so we can impute these null values with respective placeholders..

In [12]:
combined_df['customer_name'].fillna('Unknown', inplace=True)
combined_df['customer_email'].fillna('unknown@gmail.com', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_6704\2467316487.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['customer_name'].fillna('Unknown', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_6704\2467316487.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


For null values in `city` and `country`, we will impute it with a rule..

RULE ENGINE 1. `Find the most common city for that particular country and fill the null values with that city name and the same we will do for country as well with the help of city values`
BUT WHY?🤔
Key reasons for why it will be optimal:

1. `Random imputation` or global `mode imputation` could introduce unrealistic combinations `(e.g., "New York" with "India")`.

2. This method `minimizes the chance of such mismatches by using grouped statistics`.

3. `City-country pairs are naturally relational`. Filling based on known distributions respects this hierarchy, reducing logical inconsistency.


In [13]:
most_common_city = (combined_df.dropna(subset=['city']).groupby('country')['city'].agg(lambda x: x.value_counts().idxmax()))

In [14]:
print("Most common city for each country:\n")
print(most_common_city)

Most common city for each country:

country
Afghanistan                  Davidton
Albania                      New Mary
Algeria              Port Christopher
American Samoa            South David
Andorra                   North David
                           ...       
Wallis and Futuna          East Laura
Western Sahara            Laurenshire
Yemen                    South Daniel
Zambia                     East David
Zimbabwe                   West James
Name: city, Length: 243, dtype: object


In [15]:
def fill_city(row):
    if pd.isna(row['city']) and pd.notna(row['country']):
        return most_common_city.get(row['country'], None)
    return row['city']

combined_df['city'] = combined_df.apply(fill_city, axis=1)

In [16]:
most_common_country = (combined_df[combined_df['country'].notnull()].groupby('city')['country'].agg(lambda x: x.mode()[0]))  

In [17]:
print("Most common country for each city:\n")
print(most_common_country)

Most common country for each city:

city
Aaronberg                                              Bangladesh
Aaronborough                                             Barbados
Aaronburgh                                                 Israel
Aaronbury                                                 Armenia
Aaronchester                                              Reunion
                                      ...                        
Zunigaside                                                  Kenya
Zunigaton                                                Portugal
Zunigatown                                            New Zealand
Zunigaview                                                 Canada
Zunigaville     British Indian Ocean Territory (Chagos Archipe...
Name: country, Length: 59208, dtype: object


In [18]:
mask_country = combined_df['country'].isnull() & combined_df['city'].notnull()

In [19]:
combined_df.loc[mask_country, 'country'] = combined_df.loc[mask_country, 'city'].map(most_common_country)

In [20]:
combined_df[combined_df['city'].notnull() &combined_df['country'].isnull()]

,invoice_id,invoice_date,product_id,product_name,product_category,price,quantity,payment_method,customer_id,customer_name,...,city,country,inventory_id,stock_available,restock_date,supplier_id,supplier_name,supplier_email,supplier_phone,payment_id
298,81414c79-fe6a-4a36-8187-cd0b1102279d,2023-12-13,0e34e8ef-1442-46ac-b714-af2ee742ac83,Control Toys,Toys,5729.63,100.0,Bank Transfer,6af6f9f4-fd24-45da-81a0-048ef5d10bb6,Daniel Alvarado,...,West Juliaview,NaN,4ea852b8-b7af-426d-b8e4-aefbc13373ec,73.0,2025-07-14,7b3e8a35-8171-4bb1-a94b-997368f7fe19,Kerr-Douglas,lreed@brooks.com,1477543311339.0,NaN
681,520a98da-d4ae-468c-9101-ae183a98ee0b,2025-01-10,ef71549b-c7d6-4ee2-91f2-095a0746572b,Reality Electronics,Clothing,7842.47,50.0,UPI,34bdb255-385c-4799-829a-a6dccd27bece,Ryan Crawford,...,South Calvinborough,NaN,2cf44140-8463-4b0a-bd20-7408cb4960ab,7.0,2025-07-12,88aeb4b0-87d7-4d9d-9061-2bd0daab00a3,"Powell, Jenkins and Cisneros",derek65@todd.com,9967199328166.0,TQXMSC
1037,c4154837-445d-4f2d-95a4-678d19f9b4de,2025-03-31,1487b0d0-7d2e-4ac3-a28c-2c2efaffc170,Use Toys,Groceries,5185.09,165.0,Debit Card,63865aee-c33c-4b8d-89bf-bc42a82673fa,Melissa Eaton,...,North Antoniomouth,NaN,8977be0a-c833-4f20-b4bc-47785826db87,51.0,2025-07-24,ca5af0d1-3693-40db-ad84-ac48b4b48ed3,Stephens and Sons,amartin@fritz-salazar.com,437328185776.0,453245036596
1247,83d99c62-26f0-4b3c-b41c-a3c0a74e86aa,2024-04-26,fe5019a6-ed4c-444c-9249-c271d3543efc,Dark Toys,Electronics,1379.40,167.0,Bank Transfer,3eff156e-0847-4dab-a9c5-7f44d340ccb1,Rick Reed,...,North Dorothymouth,NaN,6acee528-d64b-43ac-8fed-2d44b2d64905,70.0,2025-06-19,a04a6187-90b6-43c4-9bb0-da26e244b0fd,Smith-Young,johnrhodes@parker.com,NaN,NaN
1555,25b5d749-19b2-4303-9b06-33d64716f688,2024-04-08,540ffad4-24a9-439b-acc1-2824d8be5034,Newspaper Books,Groceries,3308.80,78.0,Bank Transfer,f54db925-83db-463a-9b4c-1e8dc3ad3925,Katherine Shaw,...,North Paulaside,NaN,68755ef0-4fcb-448b-88c4-ebdeb7bf9ad1,66.0,2025-06-04,014377d7-2bbb-4deb-ae42-fec504161f5f,Hebert-Terry,rogersbill@hebert-zimmerman.com,6092616417416.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307900,42f2ef3d-5427-404b-a9ef-ef4ede69a624,2024-12-21,3da31420-4a31-480d-8a8e-5505be3f54ff,Federal Books,Groceries,190.61,145.0,UPI,1bfa6974-1fae-4105-ab53-82cfc55cfdbb,Angelica Meyer,...,South Teresastad,NaN,10aa6c84-c800-4646-ba0e-5e905389f573,28.0,2025-06-12,71b30df6-0bf4-4371-9944-cc4e25ffccbd,Mccall-Berry,williamortiz@bowers.com,9879669944322,XJYFWU
308210,b8284ed4-88a5-4b83-aead-5c4682432e62,2023-12-16,2f7bf052-96a4-4a54-bb01-c8c3edf2f614,Provide Books,Toys,9291.93,119.0,Credit Card,c7b75b9e-d172-4e93-8b98-d93c12291b8b,Eric Gomez,...,Moyerton,NaN,0ddce0fd-6b6d-4e48-895c-750073355b90,32.0,2025-05-30,e07bfe1d-0b1a-4f7b-aa22-2f405c3984ee,Price Inc,williamsmelanie@wright-allen.net,2251334043092,366035931092
309296,48171031-a85b-4937-b0f1-9fcb74638bcc,2024-03-05,1e7e98ff-28de-4386-bb01-7a7e568fac4b,Strategy Clothing,Groceries,764.07,97.0,Debit Card,dd115858-24a9-4b1d-83f2-515a0a93779e,Michael Conner,...,Summerton,NaN,c0330347-2aae-4f9d-b4c8-66b1e0a5db62,78.0,2025-06-09,682eec44-86be-4152-92c3-b98e1461ca81,Snyder LLC,scottmontes@harding.net,5546044091000,349364112329
309640,23c05c16-9830-463f-8f33-af70a9164e2e,2024-02-21,082fbe98-7146-4285-b23b-d4c47e39e06a,Indicate Clothing,Electronics,1141.09,64.0,Bank Transfer,b404517f-055a-44e6-a93f-a84b15c4d53f,Andrew Bean,...,North Jessestad,NaN,b7c2407b-0ba6-4361-86b3-04b40affcd2e,60.0,2025-07-03,e15f7d8e-72e6-4e17-b4e5-b18b3837b9bf,Williams-Kelly,mariah12@flowers.biz,4894328914590,N/A


😲 `ANOMALY AGAIN`..
If we have already imputed the city and country values above. then why these 693 rows are still there which contains NULL values?🤔

Possible reasons for it..
1. city and country `both the columns are NULL` in some rows..
2. There are `unique cities` and their country values are missing..


In [21]:
combined_df['city'].fillna('Unknown', inplace=True)
combined_df['country'].fillna('Unknown', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_6704\2024404443.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['city'].fillna('Unknown', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_6704\2024404443.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

For columns like `payment_methods`. we may impute it with the help of price columns and a real life rule.

RULE ENGINE 2. `When the prices are a multiple of 10 and is less than 3000. Customer may prefer to give cash as it won't create problems of change for them. However, if the prices are higher than 3000 like 3857, 5982, 7204 etc. Customer usually don't carry that much cash with them and should prefer upi or card payments.`

The same we have applied for this column.

In [22]:
def impute_payment_method(row):
    if pd.isnull(row['payment_method']):
        if row['price'] < 3000:
            if row['price'] %10 == 0:
                return 'cash'
            else:
                return 'upi'
        else:
            return 'card'
    return row['payment_method']

combined_df['payment_method'] = combined_df.apply(impute_payment_method, axis=1)

For the column `stock_available` which is necessary for analysis and will affect the `restock_date`. We will apply a new rule in this..

RULE ENGINE 3. As per the company policy, `All the products are equally inportant`. Since the `max stock available` in our inventory for `each product` will be `30,000 only`. So, if the stock available for every particular product is `less than 3000`. value will be `low_stock` else, ample amount of stock is available for let's say `2 months`, We can fill it with the `mean values`..

In [23]:
combined_df['total_stock_per_product'] = combined_df.groupby('product_name')['stock_available'].transform(
    lambda x: pd.to_numeric(x, errors='coerce').sum()
)
combined_df['mean_stock_per_product'] = combined_df.groupby('product_name')['stock_available'].transform(
    lambda x: np.abs(pd.to_numeric(x, errors='coerce')).mean()
)
combined_df['stock_available'] = np.where(
    combined_df['stock_available'].isna(), 
    np.where(
        combined_df['total_stock_per_product'] < 3000, 
        'low stock',  
        combined_df['mean_stock_per_product'] 
    ),
    combined_df['stock_available']  
)

combined_df = combined_df.drop(['total_stock_per_product', 'mean_stock_per_product'], axis=1)

For the column `restock_date` as it depends on `stock_available` column. Again we will apply a rule as per the policy of the company.

RULE ENGINE 4. `If the value is low stock, We need those particular products urgently. So, we will assign 2 days as the new restock date(1 for shipment and 1 for delivery respectively).`

`Also, if the stock is in between 3000 to 5000. We will assign 1 week aposterior as the new restock date.`
`Similarly, for stock in between 5000 to 10,000. We will assign 10 days aposterior as the new restock date and if it's greater than that, We have ample amount. So, 1 month aposterior will be the new restock date.`


In [24]:
combined_df['stock_available_numeric'] = pd.to_numeric(combined_df['stock_available'], errors='coerce')
today = datetime.now().date()
combined_df['restock_date'] = np.where(
    combined_df['stock_available'] == 'low stock',
    today + timedelta(days=2),  
    np.where(
        (combined_df['stock_available_numeric'] >= 3000) & (combined_df['stock_available_numeric'] < 5000),
        today + timedelta(weeks=1), 
        np.where(
            (combined_df['stock_available_numeric'] >= 5000) & (combined_df['stock_available_numeric'] < 10000),
            today + timedelta(days=10),  
            today + timedelta(days=30)   
        )
    )
)
combined_df = combined_df.drop(columns=['stock_available_numeric'])

For the column `supplier_name`, we can take help from the `supplier_email` column as users usually write their names in their mail ID's. Also the supplier phone number can be imputed with some placeholders.

In [25]:
combined_df['supplier_name'] = (
    combined_df['supplier_email']
    .str.split('@').str[0]          
    .str.replace(r'[0-9]', '', regex=True) 
)
combined_df['supplier_phone'] = combined_df['supplier_phone'].fillna('unknown')

In [26]:
combined_df.isnull().sum()

invoice_id              0
invoice_date            0
product_id              0
product_name            0
product_category        0
price                   0
quantity                0
payment_method          0
customer_id             0
customer_name           0
customer_age            0
customer_gender         0
customer_email          0
city                    0
country                 0
inventory_id            0
stock_available         0
restock_date            0
supplier_id             0
supplier_name           0
supplier_email          0
supplier_phone          0
payment_id          56547
dtype: int64

# NULL values handling completed✅. Time for some Editing🛠️.

In [27]:
combined_df.sample(5)

,invoice_id,invoice_date,product_id,product_name,product_category,price,quantity,payment_method,customer_id,customer_name,...,city,country,inventory_id,stock_available,restock_date,supplier_id,supplier_name,supplier_email,supplier_phone,payment_id
77976,90bce628-3b8e-44cf-8a9e-1eb7a398b65b,2024-03-06 00:00:00,17196558-6323-4e50-b924-f869d758a793,Respond Groceries,Clothing,88.76,14.0,UPI,57ef1529-5a01-48eb-b446-0b01c21c6d84,Dominique Gray,...,North Justin,Rwanda,6d994963-8ef3-43e3-8dea-5df59751e5ed,85.0,2025-06-25,2328c47a-2e5a-4b3d-9a92-7582ff500db9,michael,michael27@anderson-alvarez.net,5903514911270.0,ICOOTM
136079,41c0040d-6232-4263-8b90-e9244ade6e9e,2024-06-04 00:00:00,49238d9f-13a2-4596-85d9-62b667e1d658,Lose Toys,Clothing,2247.87,110.0,UPI,e50a4056-a92d-4783-a8fc-d301a15f2f10,Ann Dawson,...,Paulview,Jersey,5a2d1fd6-ee04-487d-9bdf-ec4cccc432ff,49.0,2025-06-25,4d330ab6-4d64-4bca-8974-f7f99b394948,fredchung,fredchung@young.info,6080617366397.0,VJEEMJ
190689,6f6559c3-3402-417f-ba1f-3e4ecf1d3ede,1731542400000.0,aa977ce9-0738-41d0-b767-91fe3a4c7493,Seek Groceries,Groceries,8310.74,138.0,Debit Card,6f925185-2bfc-47d7-92ac-58e54bd6232c,Michael Collier,...,Lake Jeffreychester,Albania,adac6004-5fb9-41c9-abb0-380ae478d984,7.0,2025-06-25,c9f45d2c-3030-473b-a6d7-dbe3fae5c56b,rcruz,rcruz@wolfe-woods.net,2222498191793.0,502026737234
227467,bc06ff32-43a4-43be-8ca0-1e8ac3ce2aee,1728345600000.0,fb14db6f-216d-4559-af05-324190f1eac1,Range Toys,Clothing,1823.84,55.0,UPI,fb900453-51f8-4bc9-98f0-603c591aac89,John Figueroa,...,Baileybury,Netherlands,4f3118e5-447f-4e59-b96b-ec9ef04e77eb,52.0,2025-06-25,ea07184d-adb8-4a6d-a8ac-93054c14a296,sbuck,sbuck@espinoza.com,9663854009537.0,ERSRQC
248512,45a4f911-59ba-4184-a6f2-91ecde98a7e7,2023-09-23,8a86049c-61bc-4f73-b76e-a524dff74ce4,Sign Clothing,Groceries,6879.83,24.0,Debit Card,f6886c92-fda0-4fcf-81a7-a80173a5ba58,Denise Bates,...,Lake Heather,Puerto Rico,fc17bdae-69a5-4c50-aa22-c65691e5a06f,49.0,2025-06-25,a59eca6e-e45b-42d6-862d-34d4a948b0ea,juliedunn,juliedunn@king.com,3889504432289,379898773386


Here, we have 5 columns named `invoice_id`,`inventory_id`,`customer_id`,`supplier_id` and `product_id` which contains 32 alphanumeric characters. However, since each of it is unique, we can take just the 5 characters of it to store as it will be easier to identify and more readable. Also the columns like `invoice_date` and `supplier_phone`contains some invalid values as shown above. So, we must have to perform some editing to get the valid data. 

In [28]:
cols_to_trim = ['invoice_id', 'inventory_id', 'customer_id', 'supplier_id','product_id']
for col in cols_to_trim:
    combined_df[col] = combined_df[col].astype(str).str[:5]

In [29]:
combined_df['supplier_phone'] = combined_df['supplier_phone'].astype(str).str.replace(r'\.0$', '', regex=True)

In [30]:
mask = pd.to_numeric(combined_df['invoice_date'], errors='coerce') > 1e12
combined_df.loc[mask, 'invoice_date'] = pd.to_datetime(combined_df.loc[mask, 'invoice_date'].astype(float), unit='ms')
combined_df['invoice_date'] = pd.to_datetime(combined_df['invoice_date']).dt.date

Here, the column `payment_id` is sensitive and should not be exposed to someone other than the authorized persons. Hence, we must apply some encrypting techniques to encode these credentials..

In [31]:
def mask_payment_data(df, payment_id_col='payment_id', payment_method_col='payment_method'):
    df = df.copy()
    card_mask = df[payment_method_col].isin(['Credit Card', 'Debit Card', 'Card'])
    df.loc[card_mask, payment_id_col] = df.loc[card_mask, payment_id_col].apply(
        lambda x: f"XXXX-XXXX-{str(x)[-4:]}" if pd.notna(x) and len(str(x)) >= 12 else x
    )
    upi_mask = df[payment_method_col] == 'UPI'
    df.loc[upi_mask, payment_id_col] = df.loc[upi_mask, payment_id_col].apply(
        lambda x: f"XXXX{str(x)[-2:]}" if pd.notna(x) and len(str(x)) >= 4 else x
    )
    
    return df
combined_df = mask_payment_data(combined_df)

In [32]:
combined_df.sample(5)

,invoice_id,invoice_date,product_id,product_name,product_category,price,quantity,payment_method,customer_id,customer_name,...,city,country,inventory_id,stock_available,restock_date,supplier_id,supplier_name,supplier_email,supplier_phone,payment_id
97703,2e971,2024-02-15,412ba,Office Groceries,Clothing,9718.73,124.0,UPI,ae839,Dennis Reese,...,Johnsonmouth,Zambia,13a3f,4.0,2025-06-25,5884f,mary,mary77@jacobs.biz,5959843616424,XXXXQL
46182,3bfbb,2023-08-21,a55e2,Above Clothing,Toys,8497.53,6.0,Cash,2cccf,Heather Green,...,West Kathyport,Guinea,34e70,16.0,2025-06-25,7d32f,connershannon,connershannon@hodge.com,unknown,NaN
14232,37992,2024-03-05,f894e,Along Toys,Electronics,3283.21,14.0,Debit Card,e9c4a,Yolanda Perez,...,Jonathanborough,Belize,ba589,34.0,2025-06-25,6e51f,qperry,qperry@swanson-miller.com,9668525301159,XXXX-XXXX-3150
193299,c27a4,2025-01-08,c0ad8,Water Books,Clothing,6961.64,20.0,Cash,e7412,Laura Jones,...,Catherineshire,Norfolk Island,ad19d,86.0,2025-06-25,c2f73,jcrosby,jcrosby@johnson.info,9092501191728,N/A
242237,ff39e,2024-06-12,3ae78,Or Clothing,Books,2423.47,100.0,Cash,13c67,Justin Whitaker,...,Jeffreyview,Guatemala,66d12,19.0,2025-06-25,41690,christopher,christopher81@gonzalez.com,3550714178279,N/A


In [ ]:
combined_df.shape

(272029, 23)

# 3. Normalization to eliminate Redundancy and ensuring Data Integrity..🔢

Extracting the respective columns from our dataframe to create `fact` and `dimension` tables respectively.
Here, some fields contains `NaN` which will create problem while inserting data. so, we will replace them with `None`

In [80]:
customer_dim = combined_df[['customer_id', 'customer_name', 'customer_age','customer_gender', 'customer_email', 'city', 'country']].drop_duplicates()
product_dim = combined_df[['product_id', 'product_name', 'product_category']].drop_duplicates()
supplier_dim = combined_df[['supplier_id', 'supplier_name', 'supplier_email','supplier_phone']].drop_duplicates()
payment_dim = combined_df[['payment_id', 'payment_method']].drop_duplicates().replace({np.nan :None})
inventory_dim = combined_df[['inventory_id', 'stock_available', 'restock_date']].drop_duplicates()

date_dim = combined_df[['invoice_id', 'invoice_date']].drop_duplicates()
date_dim['invoice_date'] = pd.to_datetime(date_dim['invoice_date'])
date_dim['day'] = date_dim['invoice_date'].dt.day
date_dim['month'] = date_dim['invoice_date'].dt.month
date_dim['year'] = date_dim['invoice_date'].dt.year
date_dim['quarter'] = date_dim['invoice_date'].dt.quarter


In [146]:
transaction_fact = combined_df[['invoice_id', 'invoice_date', 'product_id', 'customer_id','inventory_id', 'supplier_id', 'payment_id', 'price', 'quantity']]
# You might want to add a calculated column for total amount\
transaction_fact['payment_id'] = transaction_fact['payment_id'].where(
    pd.notnull(transaction_fact['payment_id']), None
)
transaction_fact['total_amount'] = transaction_fact['price'] * transaction_fact['quantity']

C:\Users\USER\AppData\Local\Temp\ipykernel_6704\1331200597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_fact['payment_id'] = transaction_fact['payment_id'].where(
C:\Users\USER\AppData\Local\Temp\ipykernel_6704\1331200597.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_fact['total_amount'] = transaction_fact['price'] * transaction_fact['quantity']


In [82]:
payment_dim['payment_method'] = payment_dim['payment_method'].str.upper() 

In [89]:
payment_dim['payment_method'].value_counts()

payment_method
CREDIT CARD      9949
DEBIT CARD       9939
UPI               678
CASH                3
BANK TRANSFER       2
CARD                2
Name: count, dtype: int64

In [92]:
transaction_fact

,invoice_id,invoice_date,product_id,customer_id,inventory_id,supplier_id,payment_id,price,quantity,total_amount
0,b3ce4,2024-02-04,07531,8b031,3aadf,1ca04,XXXXVJ,2361.06,152.0,358881.12
3,575af,2025-04-10,39dfe,bef40,39118,4eddb,XXXXPJ,6769.16,180.0,1218448.80
4,fb084,2025-03-25,9b7c3,be07e,c88f0,ca82e,XXXXZX,3503.24,153.0,535995.72
5,780ca,2023-12-01,144aa,22976,eb5c3,58317,XXXX-XXXX-9265,6604.15,105.0,693435.75
6,9ca15,2024-12-25,71aa1,8217a,3a838,56a6f,XXXX-XXXX-1864,1463.26,92.0,134619.92
...,...,...,...,...,...,...,...,...,...,...
309995,0f634,2023-09-07,28e7f,5c3c3,ce179,92949,XXXX-XXXX-5914,5119.57,69.0,353250.33
309996,af14d,2025-03-16,c7630,db77e,d88ab,718df,N/A,9219.54,180.0,1659517.20
309997,c4db8,2024-05-25,9ca85,be1d1,5fbe0,b7b47,XXXX-XXXX-9431,4987.59,135.0,673324.65
309998,c7ba8,2024-10-15,940ef,65ccd,94e22,b8bbb,XXXXYG,8187.13,81.0,663157.53


| Dimension       | Current NF | Why                                 
| --------------- | ---------- | ----------------------------------- 
| `customer_dim`  | **3NF**    | No transitive dependencies, all attributes depend only on customer_id    
| `product_dim`   | **3NF**    | All non-key attributes depend on the whole primary key (product_id)
| `supplier_dim`  | **3NF**    | All attributes depend entirely on supplier_id               
| `payment_dim`   | **3NF**    | Minimal table with direct dependency on payment_id                     
| `inventory_dim` | **3NF**    | Simple structure with single dependency on inventory_id                   
| `date_dim`      | **2NF**    | FDs on `invoice_date`, not PK       
| `transaction_fact`| **3NF**  | All non-key attributes depend on the combination of foreign keys

Connecting with mySQL database to create and insert data..

In [ ]:
import mysql.connector

# Define the connection
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Kok@3130",
    database="lol"
)
cursor = connection.cursor()

Design of `CUSTOMER` schema

In [2]:
def create_and_insert_customer():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dim_customer (
            customer_id VARCHAR(20) PRIMARY KEY,
            customer_name VARCHAR(100),
            customer_age INT,
            customer_gender VARCHAR(10),
            customer_email VARCHAR(100),
            city VARCHAR(100),
            country VARCHAR(100)
        )
    """)
    for _, row in customer_dim.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO dim_customer VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, tuple(row))


Design of `PRODUCT` schema

In [3]:
def create_and_insert_product():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dim_product (
            product_id VARCHAR(20) PRIMARY KEY,
            product_name VARCHAR(100),
            product_category VARCHAR(50)
        )
    """)
    for _, row in product_dim.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO dim_product VALUES (%s, %s, %s)
        """, tuple(row))


Design of `SUPPLIER` schema

In [4]:
def create_and_insert_supplier():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dim_supplier (
            supplier_id VARCHAR(20) PRIMARY KEY,
            supplier_name VARCHAR(100),
            supplier_email VARCHAR(100),
            supplier_phone VARCHAR(20)
        )
    """)
    for _, row in supplier_dim.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO dim_supplier VALUES (%s, %s, %s, %s)
        """, tuple(row))


Design of `PAYMENT` schema

In [5]:
def create_and_insert_payment():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dim_payment (
            payment_id VARCHAR(20) PRIMARY KEY,
            payment_method VARCHAR(50)
        )
    """)
    for _, row in payment_dim.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO dim_payment VALUES (%s, %s)
        """, tuple(row))


Design of `INVENTORY` schema

In [6]:
def create_and_insert_inventory():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dim_inventory (
            inventory_id VARCHAR(20) PRIMARY KEY,
            stock_available INT,
            restock_date DATE
        )
    """)
    for _, row in inventory_dim.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO dim_inventory VALUES (%s, %s, %s)
        """, tuple(row))


Design of `DATE` schema

In [8]:
def create_and_insert_date():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dim_date (
            date_id VARCHAR(20) PRIMARY KEY,
            invoice_date DATE,
            day INT,
            month INT,
            year INT,
            quarter INT
        )
    """)
    for _, row in date_dim.iterrows():
        cursor.execute("""
    INSERT IGNORE INTO dim_date (date_id, invoice_date, day, month, year, quarter)
    VALUES (%s, %s, %s, %s, %s, %s)
""", tuple(row))


Design of `FACT` schema

In [9]:
def create_and_insert_fact_sales():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS fact_sales (
            id INT AUTO_INCREMENT PRIMARY KEY,
            invoice_id VARCHAR(20),
            product_id VARCHAR(20),
            customer_id VARCHAR(20),
            supplier_id VARCHAR(20),
            payment_id VARCHAR(20),
            price DECIMAL(10,2),       
            quantity INT,
            FOREIGN KEY (product_id) REFERENCES dim_product(product_id),
            FOREIGN KEY (payment_id) REFERENCES dim_payment(payment_id),
            FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id),
            FOREIGN KEY (supplier_id) REFERENCES dim_supplier(supplier_id)
        )
    """)

    required_columns = ['invoice_id', 'product_id', 'customer_id', 'supplier_id', 'quantity', 'price', 'payment_id']
    fact_data = transaction_fact[required_columns]

    for _, row in fact_data.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO fact_sales (invoice_id, product_id, customer_id, supplier_id, quantity, price, payment_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, tuple(row))


# 4. LOAD phase

In [ ]:
create_and_insert_customer()
print("🧑‍💼 customer schema loaded successfully!")
create_and_insert_product()
print("📦 product schema loaded successfully!")
create_and_insert_supplier()
print("🚚 supplier schema loaded successfully!")
create_and_insert_payment()
print("💳 payment schema loaded successfully!")
create_and_insert_inventory()
print("🏬inventory schema loaded successfully!")
create_and_insert_date()
print("📅date schema loaded successfully!")
create_and_insert_fact_sales()
print("📈fact schema loaded successfully!")

print("✅ All data inserted into MySQL successfully.")


🧑‍💼 customer schema loaded successfully!
📦 product schema loaded successfully!
🚚 supplier schema loaded successfully!
💳 payment schema loaded successfully!
🏬inventory schema loaded successfully!
📅date schema loaded successfully!
📈fact schema loaded successfully!
✅ All data inserted into MySQL successfully.
